# Projeto 2.2 - Criar consumidor de dados para consumir o que o produtor gerar

## Preparar ambiente

In [ ]:
!pip install boto3 #lib para trabalhar com serviços AWS

In [ ]:
!pip install python-dotenv #lib para trabalhar com variáveis de ambiente no python

In [ ]:
# Importa um método da biblioteca dotenv
from dotenv import load_dotenv
import os

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

In [ ]:
# salva as variáveis de ambiente necessárias para o projeto
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
region_name = os.environ.get('REGION_NAME')
partition_key= os.environ.get('PARTITION_KEY')
shard_id = os.environ.get('SHARD_ID')

## Cria o consumidor de dados gerados pelo produtor

In [ ]:
import boto3

# criar um cliente do boto3 para o kinesis, definindo as chaves de acesso e a região
cliente = boto3.client('kinesis', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key,
                      region_name=region_name)

# criar uma variável shard que irá armazenar as informações do shard e pegar os últimos (latest) dados gerados pelo produtor
shard = cliente.get_shard_iterator(StreamName = 'stream1',
                                  ShardId = shard_id,
                                  ShardIteratorType = 'LATEST'
                                  )["ShardIterator"]

# enquanto o shard estiver ativo
while shard is not None:
    resultado = cliente.get_records(ShardIterator = shard) # pega o resultado gerado pelo produtor
    registros = resultado['Records'] # pega os registros de dados no resultado (os dados criados propriamente ditos)
    shard = resultado["NextShardIterator"] # itera o shard para continuar o loop
    
    for registro in registros: # printa as informações do registro gerado pelo produtor
        print(registro["SequenceNumber"])
        print(registro["ApproximateArrivalTimestamp"])
        print(registro["PartitionKey"])
        print(registro["Data"])